

## Chapter 7: Building a GraphQL Server

We have mastered the language of GraphQL—queries, mutations, and subscriptions. Now, we transition from consumer to architect. In this chapter, we will build a production-ready GraphQL server from scratch using Node.js.

### 7.1 Choosing a Server Framework

While you can build a GraphQL server with raw HTTP handlers, it is highly recommended to use a dedicated framework that handles parsing, validation, and execution for you.

*   **Apollo Server:** The industry standard. It is flexible, supports every major Node.js HTTP framework (Express, Fastify, Koa), and handles advanced features like file uploads and subscriptions out of the box.
*   **GraphQL Yoga:** A modern, batteries-included server built on top of `envelop` and `graphql-js`. It is excellent for quick setup and comes with built-in support for subscriptions and file uploads without extra configuration.
*   **Express-GraphQL (Legacy):** The original reference implementation by the GraphQL team. It is now deprecated in favor of Apollo Server or Yoga.

**Recommendation:** For this handbook, we will use **Apollo Server 4** due to its massive ecosystem and widespread industry adoption.

### 7.2 Project Structure for Scalability

A common mistake is putting the entire schema and resolvers in a single `index.js` file. As your API grows, this becomes unmanageable.

**Recommended Structure:**

```text
src/
├── index.js           # Entry point, server initialization
├── schema.graphql     # The Schema Definition Language file
├── resolvers/         # Folder for all resolver logic
│   ├── index.js       # Merges all resolvers
│   ├── user.js        # Resolvers for User type
│   └── query.js       # Resolvers for Query type
├── models/            # Database models (Mongoose, Sequelize, etc.)
├── datasources/       # Classes for fetching data (REST APIs, etc.)
└── context.js         # Context creation logic
```

### 7.3 The Anatomy of a Resolver

A **Resolver** is a function that is responsible for populating the data for a single field in your schema. Whenever a client queries a field, the GraphQL engine calls the corresponding resolver function to get the value.

#### 7.3.1 Parent, Args, Context, and Info

Every resolver function receives four arguments in a specific order:

```javascript
(parent, args, context, info) => { ... }
```

Let's break them down with code examples.

**The Schema:**
```graphql
type Query {
  user(id: ID!): User
}

type User {
  id: ID!
  name: String!
  friends: [User!]!
}
```

**1. `parent` (or `root`)**
This is the return value of the previous resolver in the execution chain.
*   **Example:** When resolving `Query.user`, `parent` is usually `undefined` (or the root object).
*   **Example:** When resolving `User.name`, `parent` is the actual `User` object returned by `Query.user`.

```javascript
const resolvers = {
  Query: {
    // parent is undefined here
    user: (parent, { id }) => {
      return getUserFromDatabase(id); // returns { id: "1", name: "Alice", friends: [...] }
    }
  },
  User: {
    // parent is the user object { id: "1", name: "Alice" }
    name: (parent) => {
      return parent.name.toUpperCase(); // We can transform the data
    },
    // parent is the user object
    friends: (parent) => {
      // We can use data from parent to fetch related data
      return getFriends(parent.id); 
    }
  }
};
```

**2. `args`**
An object containing all arguments passed into the field in the query.
*   **Example:** In `user(id: "123")`, `args` is `{ id: "123" }`.

```javascript
Query: {
  user: (parent, args) => {
    console.log(args.id); // "123"
    return getUser(args.id);
  }
}
```

**3. `context`**
A shared object accessible by every resolver in the query chain. This is where you put state that should be available everywhere, such as:
*   The currently logged-in user.
*   Database connections.
*   API clients (DataSources).

```javascript
// In index.js (Server setup)
const server = new ApolloServer({
  typeDefs,
  resolvers,
  context: ({ req }) => {
    // Get the user token from the headers
    const token = req.headers.authorization || '';
    
    // Try to retrieve a user with the token
    const user = getUser(token);

    // Add the user to the context
    return { user, dbConnection }; 
  },
});

// In resolvers
Query: {
  me: (parent, args, context) => {
    if (!context.user) throw new AuthenticationError('You must be logged in');
    return context.user; // Accessing context data
  }
}
```

**4. `info`**
An object containing information about the execution state of the query. It contains the raw query string, the schema definition, and the field-specific AST (Abstract Syntax Tree).
*   **Usage:** Rarely used in basic apps. It is useful for advanced features like **Lookaheads** (optimizing SQL queries based on what fields the client requested).

#### 7.3.2 Async Resolver Execution
Resolvers can return Promises. The GraphQL engine will wait for the Promise to resolve before moving to the next field.

```javascript
Query: {
  user: async (parent, { id }) => {
    // Async database call
    const user = await db.findUser(id);
    return user;
  }
}
```

### 7.4 Connecting Data Sources

In a Clean Architecture, your resolvers should **not** contain database logic. They should delegate fetching to **DataSources**. Apollo Server provides a `RESTDataSource` class that is perfect for this.

#### 7.4.1 REST Data Source Integration
This pattern wraps an existing REST API in a GraphQL layer.

**Implementation:**
```javascript
// datasources/user-api.js
const { RESTDataSource } = require('@apollo/datasource-rest');

class UserAPI extends RESTDataSource {
  constructor() {
    super();
    this.baseURL = 'https://api.example.com/';
  }

  async getUser(id) {
    // This sends a GET request to https://api.example.com/users/:id
    return this.get(`users/${id}`);
  }

  async createUser(userInput) {
    return this.post('users', userInput);
  }
}

// index.js
const server = new ApolloServer({
  typeDefs,
  resolvers,
  context: () => {
    return {
      // We make the API instance available in context
      dataSources: {
        userAPI: new UserAPI(),
      },
    };
  },
});
```

**Using in Resolver:**
```javascript
Query: {
  user: async (_, { id }, { dataSources }) => {
    return dataSources.userAPI.getUser(id);
  }
}
```

#### 7.4.2 Database Integration (SQL vs. NoSQL)
For direct database access, you typically create models using an ORM (like Prisma or Sequelize) or an ODM (like Mongoose).

**Snippet (Mongoose):**
```javascript
// models/User.js
const mongoose = require('mongoose');
const UserSchema = new mongoose.Schema({ name: String, email: String });
module.exports = mongoose.model('User', UserSchema);

// resolvers/user.js
const User = require('../models/User');

const resolvers = {
  Mutation: {
    createUser: async (_, { name, email }) => {
      const user = new User({ name, email });
      await user.save();
      return user;
    }
  }
};
```

### 7.5 The Context Object: Passing User and Request Data

As seen in section 7.3.1, the `context` function is defined at server initialization. It runs for every single request.

**Best Practice for Authentication:**
Perform authentication logic here, so you don't have to repeat it in every resolver.

```javascript
const server = new ApolloServer({
  typeDefs,
  resolvers,
  context: async ({ req }) => {
    // 1. Extract token
    const token = req.headers.authorization;
    
    // 2. Verify token (throws error if invalid)
    const user = await verifyToken(token);

    // 3. Return context
    return { user };
  }
});
```

### 7.6 Clean Architecture: Separating Business Logic from Resolvers

Resolvers should be thin glue code. They should:
1.  Validate input (basic).
2.  Call business logic.
3.  Return the result.

**Anti-Pattern:** Writing complex business rules inside a resolver makes it hard to test and reuse.

**Better Approach (Service Layer Pattern):**

```javascript
// services/user.service.js
class UserService {
  async upgradeToPremium(userId) {
    const user = await User.findById(userId);
    if (user.balance < 100) {
      throw new Error('Insufficient funds');
    }
    user.isPremium = true;
    await user.save();
    await EmailService.sendWelcome(user.email);
    return user;
  }
}

// resolvers/mutation.js
const UserService = require('../services/user.service');
const service = new UserService();

const resolvers = {
  Mutation: {
    upgradeUser: async (_, { userId }, { user }) => {
      // Resolver just orchestrates
      if (!user) throw new AuthenticationError();
      return service.upgradeToPremium(userId);
    }
  }
};
```

---

### 🚀 Next Up: Chapter 8 - Error Handling and Validation

**Summary:** Now that we have a functional server, we need to make it resilient. In Chapter 8, we will dive deep into GraphQL Error Handling. We will differentiate between syntax errors and logical errors, learn how to create custom error extensions for better debugging, and implement robust input validation strategies to protect our data integrity.